In [1]:
%reload_ext autoreload
%autoreload 2

from sarpu.data_processing import *
from sarpu.paths_and_names import *

import numpy as np
import pandas as pd
import requests

import sklearn.model_selection

# Input Data

In [2]:
# Names and locations
data_folder= "../../Data/"
data_name = "splice"
url1 = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/splice"
url2 = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/splice.t"

In [3]:
# Creation information
nb_splits = 5
test_size = 0.2

# Prepare folders

In [4]:
# Prepare folders
data_folder_original = original_data_path(data_folder,data_name)
!mkdir -p $data_folder_original
data_folder_processed = processed_data_path(data_folder,data_name)
!mkdir -p $data_folder_processed
data_folder_partitions = partitions_data_path(data_folder,data_name)
!mkdir -p $data_folder_partitions

# Download

In [5]:
#download and unzip data

unprocessed_data_path1 = os.path.join(data_folder_original,url1.split("/")[-1])
if not os.path.exists(unprocessed_data_path1):
    r = requests.get(url1, allow_redirects=True)
    open(unprocessed_data_path1, 'wb').write(r.content)

unprocessed_data_path2 = os.path.join(data_folder_original,url2.split("/")[-1])
if not os.path.exists(unprocessed_data_path2):
    r = requests.get(url2, allow_redirects=True)
    open(unprocessed_data_path2, 'wb').write(r.content)

In [6]:
#read data to pandas dataframe

nb_features=60


def line_to_example(line):
    splitline = line.split()
    cl = int(splitline[0])
    ex = eval("{"+",".join(splitline[1:])+"}")
    ex["class"]=cl
    return ex

data1 = list(map(line_to_example, filter(bool, open(unprocessed_data_path1).read().split("\n"))))
data2 = list(map(line_to_example, filter(bool, open(unprocessed_data_path2).read().split("\n"))))
data= data1+data2

df = pd.DataFrame(data)
df


,1,2,3,4,5,6,7,8,9,10,...,52,53,54,55,56,57,58,59,60,class
0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0,3.0,3.0,...,3.0,4.0,2.0,2.0,2.0,2.0,1.0,3.0,4.0,1
1,1.0,3.0,3.0,4.0,2.0,1.0,2.0,2.0,4.0,3.0,...,3.0,4.0,4.0,4.0,1.0,3.0,4.0,1.0,1.0,1
2,1.0,4.0,1.0,4.0,4.0,4.0,4.0,1.0,1.0,2.0,...,1.0,1.0,4.0,2.0,2.0,4.0,4.0,4.0,2.0,1
3,1.0,1.0,4.0,1.0,1.0,3.0,3.0,4.0,4.0,3.0,...,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,1.0,-1
4,4.0,3.0,4.0,4.0,3.0,1.0,1.0,4.0,4.0,4.0,...,3.0,2.0,1.0,4.0,2.0,1.0,1.0,4.0,4.0,-1
5,1.0,3.0,2.0,3.0,1.0,2.0,3.0,4.0,3.0,1.0,...,2.0,3.0,4.0,2.0,2.0,2.0,2.0,3.0,4.0,-1
6,1.0,4.0,2.0,1.0,4.0,2.0,4.0,4.0,4.0,2.0,...,3.0,4.0,2.0,2.0,1.0,2.0,4.0,3.0,4.0,1
7,3.0,3.0,3.0,4.0,3.0,1.0,3.0,4.0,3.0,4.0,...,2.0,1.0,1.0,4.0,3.0,3.0,1.0,2.0,1.0,1
8,2.0,1.0,3.0,4.0,3.0,3.0,1.0,1.0,4.0,2.0,...,3.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0,2.0,1
9,1.0,1.0,3.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,...,2.0,2.0,2.0,1.0,1.0,3.0,1.0,3.0,2.0,1


In [7]:
#class distribution

df["class"].value_counts()


 1    1648
-1    1527
Name: class, dtype: int64

In [8]:
# Make malignant (4) positive class

df.loc[df["class"]==-1,"class"]=0
df.loc[df["class"]==1,"class"]=1

# df

In [9]:
#normalize
for column in df.columns.values:
    df[column]=pd.to_numeric(df[column])

normalized_df=(df.astype(float)-df.min())/(df.max()-df.min())*2-1
normalized_df["class"] = df["class"]
df = normalized_df
# df

In [10]:
#move class to back

cols = list(df.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('class')) #Remove class from list
df = df[cols+['class']]

df

,1,2,3,4,5,6,7,8,9,10,...,52,53,54,55,56,57,58,59,60,class
0,-0.333333,-1.000000,-0.333333,-1.000000,-0.333333,-1.000000,-0.333333,0.333333,0.333333,0.333333,...,0.333333,1.000000,-0.333333,-0.333333,-0.333333,-0.333333,-1.000000,0.333333,1.000000,1
1,-1.000000,0.333333,0.333333,1.000000,-0.333333,-1.000000,-0.333333,-0.333333,1.000000,0.333333,...,0.333333,1.000000,1.000000,1.000000,-1.000000,0.333333,1.000000,-1.000000,-1.000000,1
2,-1.000000,1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-0.333333,...,-1.000000,-1.000000,1.000000,-0.333333,-0.333333,1.000000,1.000000,1.000000,-0.333333,1
3,-1.000000,-1.000000,1.000000,-1.000000,-1.000000,0.333333,0.333333,1.000000,1.000000,0.333333,...,0.333333,1.000000,-1.000000,0.333333,0.333333,1.000000,-0.333333,-0.333333,-1.000000,0
4,1.000000,0.333333,1.000000,1.000000,0.333333,-1.000000,-1.000000,1.000000,1.000000,1.000000,...,0.333333,-0.333333,-1.000000,1.000000,-0.333333,-1.000000,-1.000000,1.000000,1.000000,0
5,-1.000000,0.333333,-0.333333,0.333333,-1.000000,-0.333333,0.333333,1.000000,0.333333,-1.000000,...,-0.333333,0.333333,1.000000,-0.333333,-0.333333,-0.333333,-0.333333,0.333333,1.000000,0
6,-1.000000,1.000000,-0.333333,-1.000000,1.000000,-0.333333,1.000000,1.000000,1.000000,-0.333333,...,0.333333,1.000000,-0.333333,-0.333333,-1.000000,-0.333333,1.000000,0.333333,1.000000,1
7,0.333333,0.333333,0.333333,1.000000,0.333333,-1.000000,0.333333,1.000000,0.333333,1.000000,...,-0.333333,-1.000000,-1.000000,1.000000,0.333333,0.333333,-1.000000,-0.333333,-1.000000,1
8,-0.333333,-1.000000,0.333333,1.000000,0.333333,0.333333,-1.000000,-1.000000,1.000000,-0.333333,...,0.333333,-0.333333,-0.333333,-0.333333,-0.333333,-1.000000,-1.000000,0.333333,-0.333333,1
9,-1.000000,-1.000000,0.333333,-1.000000,-1.000000,0.333333,-1.000000,-1.000000,-0.333333,-1.000000,...,-0.333333,-0.333333,-0.333333,-1.000000,-1.000000,0.333333,-1.000000,0.333333,-0.333333,1


In [11]:
# make numpy array

import numpy as np

xy = df.values

x = xy[:,:-1].astype(float)
y = xy[:,-1].astype(int)

x_pos = x[y==1]
x_neg = x[y==0]



In [12]:
#Save data and true classes
np.savetxt(data_path(data_folder, data_name), x)
np.savetxt(classlabels_path(data_folder, data_name), y,fmt='%d')

# Different dataset partitions (train/test and class prior)

In [13]:
sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=nb_splits, test_size=test_size, random_state=0)
splits = list(sss.split(x,y))

In [14]:
#save partitions. 0 means not in data, 1 means in train partition, 2 means in test partition

for i, (train,test) in enumerate(splits):
    partition = np.zeros_like(y,dtype=int)
    partition[train]=1
    
    partition[test]=2    
    np.savetxt(partition_path(data_folder,data_name, i), partition, fmt='%d')
   